In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os
from datetime import datetime, timedelta

In [2]:
# Set paths
depth_file = "data/depth20_1000ms.csv"
trade_file = "data/aggTrade-tot.csv"
output_file = "data/extracted_features.csv"

print("Loading data files...")
# Load data
depth_df = pd.read_csv(depth_file)
trade_df = pd.read_csv(trade_file)

# Convert time columns to datetime with proper parsing
depth_df['Time'] = pd.to_datetime(depth_df['Time'], errors='coerce')
trade_df['Time'] = pd.to_datetime(trade_df['Time'], errors='coerce')


print(f"Loaded depth data: {depth_df.shape} rows")
print(f"Loaded trade data: {trade_df.shape} rows")

Loading data files...


C:\Users\shory\AppData\Local\Temp\ipykernel_13396\518127257.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  depth_df['Time'] = pd.to_datetime(depth_df['Time'], errors='coerce')
C:\Users\shory\AppData\Local\Temp\ipykernel_13396\518127257.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trade_df['Time'] = pd.to_datetime(trade_df['Time'], errors='coerce')


Loaded depth data: (508791, 81) rows
Loaded trade data: (933417, 6) rows


In [3]:
# Sort by time
depth_df = depth_df.sort_values('Time')
trade_df = trade_df.sort_values('Time')

In [4]:
print("Creating features...")

# Function to calculate features for each timestamp
def calculate_features(row):
    features = {}
    
    # Basic features
    mid_price = (row['BidPriceL1'] + row['AskPriceL1']) / 2
    features['mid_price'] = mid_price
    
    # 1. Liquidity Features
    # Bid-Ask spread
    features['bid_ask_spread'] = row['AskPriceL1'] - row['BidPriceL1']
    features['bid_ask_spread_bps'] = features['bid_ask_spread'] / mid_price * 10000
    
    # Order book imbalance at different levels
    for i in range(1, 21):
        bid_qty = row[f'BidQtyL{i}']
        ask_qty = row[f'AskQtyL{i}']
        features[f'imbalance_lvl{i}'] = (bid_qty - ask_qty) / (bid_qty + ask_qty) if (bid_qty + ask_qty) > 0 else 0
    
    # Microprice
    features['microprice'] = (row['BidPriceL1'] * row['AskQtyL1'] + row['AskPriceL1'] * row['BidQtyL1']) / (row['BidQtyL1'] + row['AskQtyL1']) if (row['BidQtyL1'] + row['AskQtyL1']) > 0 else mid_price
    
    # Cumulative depth
    cum_bid_qty = sum(row[f'BidQtyL{i}'] for i in range(1, 21))
    cum_ask_qty = sum(row[f'AskQtyL{i}'] for i in range(1, 21))
    features['cum_bid_qty'] = cum_bid_qty
    features['cum_ask_qty'] = cum_ask_qty
    features['cum_depth_imbalance'] = (cum_bid_qty - cum_ask_qty) / (cum_bid_qty + cum_ask_qty) if (cum_bid_qty + cum_ask_qty) > 0 else 0
    
    # Sloped depth - measure how quickly size decays away from top of book
    bid_slope = np.polyfit([i for i in range(1, 6)], [row[f'BidQtyL{i}'] for i in range(1, 6)], 1)[0]
    ask_slope = np.polyfit([i for i in range(1, 6)], [row[f'AskQtyL{i}'] for i in range(1, 6)], 1)[0]
    features['bid_slope'] = bid_slope
    features['ask_slope'] = ask_slope
    
    # Price levels calculation
    bid_price_levels = [row[f'BidPriceL{i}'] for i in range(1, 21)]
    ask_price_levels = [row[f'AskPriceL{i}'] for i in range(1, 21)]
    
    # Mean distance between levels
    bid_diffs = np.diff(bid_price_levels)
    ask_diffs = np.diff(ask_price_levels)
    features['mean_bid_price_spacing'] = np.mean(bid_diffs) if len(bid_diffs) > 0 else 0
    features['mean_ask_price_spacing'] = np.mean(ask_diffs) if len(ask_diffs) > 0 else 0
    
    # Total depth in first N levels
    for n in [5, 10, 20]:
        features[f'bid_depth_{n}lvl'] = sum(row[f'BidQtyL{i}'] for i in range(1, n+1))
        features[f'ask_depth_{n}lvl'] = sum(row[f'AskQtyL{i}'] for i in range(1, n+1))
    
    return features

# Apply function to extract base features
print("Calculating base features...")
features_list = []
for idx, row in depth_df.iterrows():
    if idx % 10000 == 0:
        print(f"Processing row {idx}/{len(depth_df)}")
    features = calculate_features(row)
    features['Time'] = row['Time']
    features_list.append(features)

base_features_df = pd.DataFrame(features_list)
print(f"Base features calculated: {base_features_df.shape}")

Creating features...
Calculating base features...
Processing row 0/508791
Processing row 10000/508791
Processing row 20000/508791
Processing row 30000/508791
Processing row 40000/508791
Processing row 50000/508791
Processing row 60000/508791
Processing row 70000/508791
Processing row 80000/508791
Processing row 90000/508791
Processing row 100000/508791
Processing row 110000/508791
Processing row 120000/508791
Processing row 130000/508791
Processing row 140000/508791
Processing row 150000/508791
Processing row 160000/508791
Processing row 170000/508791
Processing row 180000/508791
Processing row 190000/508791
Processing row 200000/508791
Processing row 210000/508791
Processing row 220000/508791
Processing row 230000/508791
Processing row 240000/508791
Processing row 250000/508791
Processing row 260000/508791
Processing row 270000/508791
Processing row 280000/508791
Processing row 290000/508791
Processing row 300000/508791
Processing row 310000/508791
Processing row 320000/508791
Process

In [5]:
# Calculate time-based features
def calculate_time_features(df):
    # Sort by time
    df = df.sort_values('Time')
    
    # Calculate returns
    df['return_1s'] = df['mid_price'].pct_change()
    
    # Rolling statistics for mid price
    for window in [5, 10, 30, 60]:
        # Volatility
        df[f'volatility_{window}s'] = df['return_1s'].rolling(window).std()
        
        # Momentum/trend indicators
        df[f'return_{window}s'] = df['mid_price'].pct_change(window)
        df[f'rsi_{window}s'] = calculate_rsi(df['mid_price'], window)
        
        # Mean reversion indicators
        df[f'zscore_{window}s'] = (df['mid_price'] - df['mid_price'].rolling(window).mean()) / df['mid_price'].rolling(window).std()
    
    # Trend detection using linear regression slope
    for window in [10, 30, 60]:
        df[f'trend_slope_{window}s'] = calculate_rolling_slope(df['mid_price'], window)
    
    return df

def calculate_rsi(prices, window):
    # Calculate RSI (Relative Strength Index)
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_rolling_slope(series, window):
    # Function to calculate rolling slope
    result = np.full_like(series, np.nan, dtype=float)
    
    for i in range(window, len(series)):
        if i >= window:
            x = np.arange(window)
            y = series.iloc[i-window:i].values
            if not np.isnan(y).any():  # Only calculate if no NaN values
                slope = np.polyfit(x, y, 1)[0]
                result[i] = slope
    
    return pd.Series(result, index=series.index)

# Apply time-based features
print("Calculating time-based features...")
features_df = calculate_time_features(base_features_df)

Calculating time-based features...


In [6]:
features_df.head()

,mid_price,bid_ask_spread,bid_ask_spread_bps,imbalance_lvl1,imbalance_lvl2,imbalance_lvl3,imbalance_lvl4,imbalance_lvl5,imbalance_lvl6,imbalance_lvl7,...,return_30s,rsi_30s,zscore_30s,volatility_60s,return_60s,rsi_60s,zscore_60s,trend_slope_10s,trend_slope_30s,trend_slope_60s
0,579.480,0.08,1.380548,0.000000,0.000000,-0.184557,0.263830,0.951269,0.236212,-0.617138,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,579.400,0.08,1.380739,0.000000,0.803564,0.264728,0.596619,0.619491,0.516410,-0.560092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,579.410,0.08,1.380715,-0.098286,0.083763,-0.193003,0.162734,0.669926,-0.359120,0.594016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,579.340,0.06,1.035661,0.000000,0.021604,-0.016347,-0.409808,-0.965007,0.006461,-0.981413,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,579.345,0.05,0.863044,0.000000,-0.298028,-0.427626,0.162734,-0.026723,-0.963121,-0.122104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Merge with trade data to calculate volume features
def add_volume_features(features_df, trade_df):
    # Create time buckets in trade data matching feature data timestamps
    trade_df_copy = trade_df.copy()
    
    unique_times = features_df['Time'].unique()
    
    volume_features = []
    
    for time in unique_times:
        # Look back periods for volume calculations
        for seconds in [10, 30, 60]:
            start_time = time - pd.Timedelta(seconds=seconds)
            
            # Filter trades in the window
            window_trades = trade_df_copy[(trade_df_copy['Time'] >= start_time) & (trade_df_copy['Time'] <= time)]
            
            # Calculate features
            if len(window_trades) > 0:
                total_volume = window_trades['Quantity'].sum()
                num_trades = len(window_trades)
                avg_trade_size = total_volume / num_trades if num_trades > 0 else 0
                
                # Calculate buy vs sell volume (M = True for buy)
                buy_volume = window_trades[window_trades['M'] == True]['Quantity'].sum()
                sell_volume = window_trades[window_trades['M'] == False]['Quantity'].sum()
                
                volume_imbalance = (buy_volume - sell_volume) / (buy_volume + sell_volume) if (buy_volume + sell_volume) > 0 else 0
                
                # VWAP
                vwap = (window_trades['Price'] * window_trades['Quantity']).sum() / window_trades['Quantity'].sum() if window_trades['Quantity'].sum() > 0 else None
            else:
                total_volume = 0
                num_trades = 0
                avg_trade_size = 0
                volume_imbalance = 0
                vwap = None
            
            volume_features.append({
                'Time': time,
                f'volume_{seconds}s': total_volume,
                f'num_trades_{seconds}s': num_trades,
                f'avg_trade_size_{seconds}s': avg_trade_size,
                f'volume_imbalance_{seconds}s': volume_imbalance,
                f'vwap_{seconds}s': vwap
            })
    
    # Convert to DataFrame
    volume_features_df = pd.DataFrame(volume_features)
    
    # Merge with main features
    merged_df = features_df.merge(volume_features_df, on='Time', how='left')
    
    return merged_df

print("Adding volume features...")
final_features_df = add_volume_features(features_df, trade_df)

Adding volume features...


In [8]:
# Calculate VWAP shift
for window in [10, 30]:
    col = f'vwap_{window}s'
    if col in final_features_df.columns:
        final_features_df[f'vwap_shift_{window}s'] = final_features_df[col].pct_change()

C:\Users\shory\AppData\Local\Temp\ipykernel_13396\1301364045.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  final_features_df[f'vwap_shift_{window}s'] = final_features_df[col].pct_change()
C:\Users\shory\AppData\Local\Temp\ipykernel_13396\1301364045.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  final_features_df[f'vwap_shift_{window}s'] = final_features_df[col].pct_change()


In [9]:
# Normalize features
def normalize_features(df):
    # Make a copy of the dataframe
    df_norm = df.copy()
    
    # Get the columns to normalize (exclude Time and any columns with NaN)
    cols_to_normalize = [col for col in df.columns if col != 'Time' and df[col].notna().all()]
    
    # Initialize the scaler
    scaler = StandardScaler()
    
    # Fit and transform the selected columns
    df_norm[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])
    
    return df_norm

print("Normalizing features...")
normalized_df = normalize_features(final_features_df)

Normalizing features...


In [10]:
# Fill NaN values
normalized_df = normalized_df.fillna(0)

# Save features
print(f"Saving features to {output_file}...")
normalized_df.to_csv(output_file, index=False)

print("Feature extraction complete!")
print(f"Total features extracted: {len(normalized_df.columns) - 1}")  # -1 for Time column

C:\Users\shory\AppData\Local\Temp\ipykernel_13396\2655837052.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  normalized_df = normalized_df.fillna(0)


Saving features to data/extracted_features.csv...
Feature extraction complete!
Total features extracted: 74
